# Análisis de Datos
## TP N 1
----
### Grupo N° 7
- Aviani, José
- Díaz, José Luis
- Silvera, Ricardo

---
## Introducción

Para este trabajo elegimos el el dataset Precios Claros – Base SEPA, perteneciente al “Sistema Electrónico de Publicidad de Precios Argentinos (SEPA)" (https://datos.gob.ar/), el cual reúne los precios de comercios minoristas (grandes establecimientos) de más de 70 mil productos en toda la Argentina. Particularmente para este trabajo, seleccionamos el set de datos del establecimiento **Carrefour** ya que era el de mayor tamaño, lo cual es deseable como entrada en un problema de aprendizaje de máquina.
A continuación realizamos el análisis exploratorio de los datos y finalizamos con las conclusiones obtenidas del trabajo.


### Importación de librerías

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## Análisis Exploratorio de los Datos

### Carga y comprensión de los datos


Los datos están separados en 3 archivos
- comercio
- sucursales
- productos

Cada uno contiene información específica sobre la entidad que indica su nombre. Vamos a investigar a cada uno para comprender la información que contienen y como está organizada y unificar todos los datos que nos interesen en un único dataframe. 

In [ ]:
#Comercio
comercio=pd.read_csv('./dataset/comercio.csv',delimiter='|')
#Sucursales
sucursales=pd.read_csv('./dataset/sucursales.csv',delimiter='|')
#Productos
productos=pd.read_csv('./dataset/productos.csv',delimiter='|',low_memory=False)

# La ultima linea del achivo es solamente la fecha de actualizacion, por eso la borramos
comercio = comercio.iloc[:-1]
sucursales = sucursales.iloc[:-1]
productos = productos.iloc[:-1]


In [ ]:
productos.info()

In [ ]:
#Visualizamos las columnas de cada Dataframe
print("Columnas de Comercio:\n",comercio.columns)
print("Columnas de Sucursales:\n",sucursales.columns)
print("Columnas de Productos:\n",productos.columns)

Unimos los 3 dataframe en uno solo

In [ ]:
comercio=comercio[['id_bandera','comercio_bandera_nombre']]
datos=pd.merge(productos,pd.merge(comercio,sucursales,on="id_bandera"),on='id_sucursal')

Eliminamos las columnas que no son de interes de análisis

In [ ]:
datos.columns

In [ ]:
#Eliminar columnas
datos=datos.drop(columns=['id_comercio_x','id_comercio_y','id_bandera_x','id_bandera_y',
        'sucursales_lunes_horario_atencion',
        'sucursales_martes_horario_atencion',
        'sucursales_miercoles_horario_atencion',
        'sucursales_jueves_horario_atencion',
        'sucursales_viernes_horario_atencion',
        'sucursales_sabado_horario_atencion',
        'sucursales_domingo_horario_atencion'])
datos.head()


In [ ]:
datos.info()

### Descripción de las columnas

El archivo contiene 2.835.684 filas con información de los productos y sucursales.

#### Itentificadores

-  `id_comercio ` : Es de tipo numérico, representa el identificador único del comercio al que pertenece el producto. Es constante para nuestro caso.

-  `id_bandera `: Se refiere al tipo de comercio, y está relacionado con el campo  `id_bandera ` del archivo de comercio.
-  `id_sucursal `: Es de tipo numérico, contiene el código de la sucursal específica dentro del comercio, se relaciona con el campo de mismo nommbre del archivo sucursales.
-  `id_producto `: Es de tipo numérico, es el identificador único del producto para el comercio.
-  `productos_ean `: Es de tipo numérico, representa el código EAN (European Article Number) del producto, utilizado como código de barras.
-  `productos_descripcion `: Es de tipo texto, contiene la descripción comercial del producto.
-  `productos_marca`: Es de tipo texto, indica el nombre de la marca comercial del producto.

#### Sucursal
- `id_sucursal`: Es de tipo numérico, es el identificador de la sucursal

- `sucursales_tipo`: Es de tipo texto, indica el tipo de establecimiento. Relacionado con la cantidad de cajas.
- `comercio_bandera_nombre`:  'Hipermercado Carrefour', 'Express' o 'Market'
- `sucursales_calle`: Es de tipo texto, representa el nombre de la calle donde está ubicada la sucursal.
- `sucursales_numero`: Es de tipo numérico o texto, indica el número de la dirección de la sucursal.
- `sucursales_latitud`: Es de tipo numérico, representa la coordenada de latitud de la ubicación geográfica de la sucursal.
- `sucursales_longitud`: Es de tipo numérico, representa la coordenada de longitud de la ubicación geográfica de la sucursal.
- `sucursales_observaciones`: Es de tipo texto, contiene observaciones o notas adicionales sobre la sucursal.
- `sucursales_codigo_postal`: Es de tipo texto o numérico, representa el código postal correspondiente a la dirección de la sucursal.
- `sucursales_localidad`: Es de tipo texto, indica la localidad o ciudad donde se encuentra la sucursal.
- `sucursales_barrio`: Es de tipo texto, indica el barrio donde se encuentra ubicada la sucursal.
- `sucursales_provincia`: Es de tipo texto, contine un código que indica la provincia a la que pertenece la sucursal.

#### Unidades de medida y cantidades

-  `productos_cantidad_presentacion `: Es de tipo numérico, indica la cantidad del producto contenida en la unidad.

-  `productos_unidad_medida_presentacion `: Es de tipo texto, representa la unidad de medida de la presentación del producto.
-  `productos_cantidad_referencia `: Es de tipo numérico, indica la cantidad de producto usada como base para calcular el precio de referencia.
-  `productos_unidad_medida_referencia `: Es de tipo texto, representa la unidad de medida de la cantidad de referencia.

#### Precios 
-  `productos_precio_lista` : Es de tipo numérico, representa el precio de lista del producto, es decir, el precio regular sin promociones.

-  `productos_precio_referencia `: Es de tipo numérico, corresponde al precio de referencia.

#### Promociones

-  `productos_precio_unitario_promo1 `: Es de tipo numérico, indica el precio unitario del producto en la primera promoción (si existe).

-  `productos_leyenda_promo1 `: Es de tipo texto, contiene la leyenda o descripción asociada a la primera promoción.
-  `productos_precio_unitario_promo2 `: Es de tipo numérico, indica el precio unitario del producto en la segunda promoción (si existe).
-  `productos_leyenda_promo2 `: Es de tipo texto, contiene la leyenda o descripción asociada a la segunda promoción.

El dataframe **comercio** contiene solo 3 filas con información, mucha de la cuales son repetitiva y constante para nuestro caso que solo estudiaremos un solo comercio.

- `id_comercio`: Es el identificador de comercio, como trabajaremos solo con Carefour este data será constante en todo el dataset.

- `id_bandera`: Es de tipo numérico,  identifica el tipo de comercio cuya discreción en texto está en el campo comercio_bandera_nombre 
- `icomercio_cui`: Es de tipo numérico, contiene el numero de CUIT del comercio, constante para nuestro caso. 
- `comercio_razon_social`:  Es de tipo texto, contiene el la Razón Social del comercio, es constante para nuestro caso.
- `comercio_bandera_nombre`: Contiene la descripción del tipo de comercio (Hipermercado Carrefeour, Express, Market). es la descripción del valor numérico del campo id_bandera.
- `comercio_ultima_actualizacion`: Contiene la fecha y hora de la actualización del archivo. Es constante para nuestro caso ya que trabajaremos con un único archivo.
- `comercio_version_sepa`: Es el numero de versión que indica la versión del modelo de datos del paquete SEPA.


Las unicas columnas que tienen sentido en este caso son `id_bandera` y `comercio_bandera_nombre`, que nos permitiran entender el dimensión del tipo de sucursal.

### Vista general del dataset (Ricardo)

### Ajustes de tipos de datos (Ricardo)

In [ ]:
### Ajustando los tipos de datos
datos['id_sucursal']=datos['id_sucursal'].astype('category')
datos['productos_unidad_medida_presentacion']=datos['productos_unidad_medida_presentacion'].astype('category')
datos['productos_unidad_medida_referencia']=datos['productos_unidad_medida_referencia'].astype('category')
datos['productos_marca']=datos['productos_marca'].astype('category')
datos['sucursales_tipo']=datos['sucursales_tipo'].astype('category')
datos['sucursales_barrio']=datos['sucursales_barrio'].astype('category')
datos['sucursales_codigo_postal']=datos['sucursales_codigo_postal'].astype('category')
datos['sucursales_localidad']=datos['sucursales_localidad'].astype('category')
datos['sucursales_provincia']=datos['sucursales_provincia'].astype('category')

### Variables categóricas: exploración (José)



In [ ]:
#Gráficos de barras para variables categóricas

### Variables numéricas: estadística descriptiva (Ricardo)

In [ ]:
#Para cada variable de abajo  
# Histogramas de variables numéricas
# Boxplots para detectar outliers

### Visualizaciones entre pares de variables (José)


Investigamos relación entre variables para descubiri dependencias o relación en ellas. Principalmente la variable precio_lista con el resto que es el target de nuestro problema
Matriz de correlación (df.corr())

Gráficos de dispersión (sns.scatterplot())

Agrupaciones y comparaciones por categorías (groupby())

## Revisión de valores faltantes (Joselo) 



In [ ]:
#Analizara para las columnas relevantes los datos faltantes

## Detección de outliers (Joselo) 

### Exportación de dataset analizado (ADAPTAR)

In [ ]:
# MODIFICAR EL CODIGO PARA NUJESTR DATASET

import json

# Detectar columnas categóricas
categorical_cols = [col for col, dtype in carreras_24.dtypes.items() if str(dtype) == 'category']

# Guardar los tipos como strings
dtypes_str = {col: str(dtype) for col, dtype in carreras_24.dtypes.items()}

# Guardar todo en un solo JSON
info = {
    "dtypes": dtypes_str,
    "categoricals": categorical_cols
}

with open("../datasets/carreras_24_1_dtypes.json", "w") as f:
    json.dump(info, f)

# exportar CSV
carreras_24.to_csv('../datasets/carreras_24_1.csv', index=False)

---
## Conclusiones